In [25]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Dataset joylashgan papka
DATASET_PATH = "diabetic"

# Parametrlar
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20

# 🔹 Ma'lumotlarni yuklash va tayyorlash
datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training"
)

val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation"
)

# 🔹 EfficientNetB0 bilan asosiy model
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Modelni muzlatish
base_model.trainable = False


Found 879 images belonging to 1 classes.
Found 219 images belonging to 1 classes.


In [28]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 🔹 Dataset joylashgan papka
DATASET_PATH = "diabetic/"  # Barcha sinflar shu papkada bo'lishi kerak

# 🔹 Model parametrlari
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25
NUM_CLASSES = 2  # 4 ta sinf (Sog‘lom, Diabetik retinopatiya, Glaukoma, To‘r parda kasalliklari)

# 🔹 Ma'lumotlarni yuklash va tayyorlash
datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",  # Ko'p sinfli klassifikatsiya uchun categorical
    subset="training"
)

val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

# 🔹 EfficientNetB0 bilan asosiy model
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Modelni muzlatish (pre-trained qatlamlar)

# 🔹 Yangi klassifikator qatlami
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
output = Dense(NUM_CLASSES, activation="softmax")(x)  # Ko‘p sinfli model uchun softmax

# 🔹 Model yaratish
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# 🔹 Modelni o‘qitish
history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS)

# 🔹 Modelni saqlash
model.save("fundus_disease_classifier.h5")


Found 950 images belonging to 2 classes.


Found 236 images belonging to 2 classes.
Epoch 1/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 22s 533ms/step - accuracy: 0.8567 - loss: 0.3861 - val_accuracy: 0.9280 - val_loss: 0.2713
Epoch 2/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 14s 476ms/step - accuracy: 0.9242 - loss: 0.2736 - val_accuracy: 0.9280 - val_loss: 0.2589
Epoch 3/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 16s 537ms/step - accuracy: 0.9288 - loss: 0.2628 - val_accuracy: 0.9280 - val_loss: 0.2593
Epoch 4/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 18s 587ms/step - accuracy: 0.9395 - loss: 0.2421 - val_accuracy: 0.9280 - val_loss: 0.2664
Epoch 5/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 16s 531ms/step - accuracy: 0.9395 - loss: 0.2445 - val_accuracy: 0.9280 - val_loss: 0.2671
Epoch 6/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 17s 569ms/step - accuracy: 0.9361 - loss: 0.2444 - val_accuracy: 0.9280 - val_loss: 0.2623
Epoch 7/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 17s 564ms/step - accuracy: 0.9269 - loss: 0.2715 - val_accuracy: 0.9280 - val_loss: 0.2616
Epoch 8/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 16s 543ms/step - a

In [30]:
# Modelni yuklash
from tensorflow.keras.models import load_model

# Modelni yuklash
model = load_model("fundus_disease_classifier.h5")

# Modelning test ma’lumotlari bo‘yicha aniqligini tekshirish
test_loss, test_acc = model.evaluate(val_data)
print(f"✅ Modelning aniqligi: {test_acc * 100:.2f}%")


8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 369ms/step - accuracy: 0.9225 - loss: 0.2744
✅ Modelning aniqligi: 92.80%


In [35]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Sinflar ro‘yxati
CLASSES = ["Diabetik retinopatiya", "Glaukoma", "To‘r parda kasalliklari"]

# Modelning taxminlarini olish
y_true = []
y_pred = []

for _ in range(len(val_data)):
    X, y = val_data.next()
    preds = model.predict(X)
    y_true.extend(np.argmax(y, axis=1))  # Haqiqiy sinflar
    y_pred.extend(np.argmax(preds, axis=1))  # Model taxmin qilgan sinflar

# Confusion Matrix hosil qilish
cm = confusion_matrix(y_true, y_pred)

# Vizualizatsiya qilish
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=CLASSES, yticklabels=CLASSES)
plt.xlabel("Model taxmini")
plt.ylabel("Haqiqiy sinf")
plt.title("Confusion Matrix")
plt.show()


AttributeError: 'DirectoryIterator' object has no attribute 'next'

In [37]:
# Classification report chiqarish
report = classification_report(y_true, y_pred, target_names=CLASSES)
print(report)


ValueError: Number of classes, 0, does not match size of target_names, 3. Try specifying the labels parameter

In [36]:

# 🔹 Yangi klassifikator qatlami
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
output = Dense(1, activation="sigmoid")(x)

# Model yaratish
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# 🔹 Modelni o‘qitish
history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS)

# 🔹 Modelni saqlash
model.save("diabetic_retinopathy_classifier.h5")

Epoch 1/25


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 2), output.shape=(None, 1)

In [24]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2

# Modelni yuklash
model = load_model("diabetic_retinopathy_classifier.h5")

# Tasvirni yuklash va tayyorlash
def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)
    return "Diabetik retinopatiya bor" if prediction > 0.5 else "Sog‘lom"

# Natija olish
print(predict_image("test_fundus.jpg"))


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'diabetic_retinopathy_classifier.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)